<a href="https://colab.research.google.com/github/MohmadYaacoub/Data/blob/main/Data_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import** **Libraries** ##

In [3]:
import glob
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from pathlib import Path
from tqdm import tqdm
import pickle as pkl
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

## **Data preparation**

In [6]:
def data_generator(base_file_path, max_files_per_class=50):
    # Retrieve all directories in the base file path
    data = glob.glob(base_file_path + "/*")

    # Generate the dictionary keys based on folder names
    category_names = [os.path.basename(directory_path) for directory_path in data]
    text_files = {category: [] for category in category_names}

    # Assign file paths to the respective categories
    for directory_path in data:
        category = os.path.basename(directory_path)
        if category in text_files:
            text_files[category].append(directory_path)

    # Initialize lists to hold the data and labels
    X, y = [], []
    class_counts = {key: 0 for key in text_files.keys()}  # Initialize class counts

    # Process each category
    for label, (key, directory_paths) in enumerate(text_files.items(), start=0):
        for directory_path in directory_paths:
            files_in_directory = glob.glob(directory_path + "/*")
            for file_path in files_in_directory:
              df = pd.read_csv(file_path, header=None)
              df_array = df.iloc[1:51, :35].values # choose the window array that is valid for your application)
              print(df_array.shape, label)
              X.append(df_array)
              y.append(label)  # Assign numeric label
              class_counts[key] += 1  # Increment class count
              # Break if reached the maximum number of files for this class
              if class_counts[key] == max_files_per_class:
                  break
            print(f"Class {key}: {class_counts[key]} files")

    # Convert lists to NumPy arrays
    X = np.asarray(X, dtype=np.float32)
    y = np.asarray(y, dtype=np.int32)

    print("X shape:", X.shape)
    print("y shape:", y.shape)

    return X, y

# add path
file_path = '/content/drive/MyDrive/Csv Fusion'
X, y = data_generator(file_path, max_files_per_class=50)


(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
(50, 35) 0
Class Umbrella: 50 files
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35) 1
(50, 35

## **Split the data**

In [7]:
X_train, x_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(x_tmp, y_tmp, test_size=0.5, random_state=42, stratify=y_tmp)

In [8]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

## **Model**

In [ ]:
class CNN_1D():
    """
    CNN_1D: Class to train a 1D Convolutional Neural Network
    Attributes:
    filters: List of filters for the convolutional layers
    kernels: List of kernel sizes for the convolutional layers
    window: Window size for the input data
    n_folds: Number of folds for cross-validation
    channels: Number of channels in the input data
    batch_size: Batch size for training
    epochs: Number of epochs for training
    learning_rate: Learning rate for the optimizer
    optimizer: Optimizer for training
    loss: Loss function for training
    metrics: Metrics for training
    callbacks: Callbacks for training
    norm_dict: Dictionary with normalization parameters
    train: Method to train the model
    """
    def __init__(self,
                 X_train,
                 y_train,
                 X_val,
                 y_val,
                 X_test,
                 y_test):
        self.filters = [[8], [16]]
        self.kernels = [2, 3, 4]
        self.batch_size = 64
        self.epochs = 300
        self.learning_rate = 0.001
        self.n_classes = 28
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        self.loss = 'categorical_crossentropy'
        self.metrics = ['accuracy']
        self.callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
                         # tf.keras.callbacks.LearningRateScheduler(schedule=lambda epoch, lr: lr * 0.5 if epoch % 5 == 0 and epoch != 0 and lr>1e-4 else lr)]

        # Assign the training, validation, and testing data
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.X_test = X_test
        self.y_test = y_test
        ########## Train ################
        self.train()

    def __create_model(self, filters, kernel, n_classes, verbose=1):
        model = tf.keras.models.Sequential()
        for i, f in enumerate(filters):
            if i == 0:
                model.add(tf.keras.layers.Conv1D(f, kernel, activation='relu', input_shape=(self.X_train.shape[1:]), padding='same'))
            else:
                model.add(tf.keras.layers.Conv1D(f, kernel, activation='relu', padding='same'))
            model.add(tf.keras.layers.BatchNormalization())
            model.add(tf.keras.layers.MaxPooling1D(2))
        model.add(tf.keras.layers.GlobalAveragePooling1D())
        # model.add(tf.keras.layers.Dense(128, activation='relu'))
        model.add(tf.keras.layers.Dense(n_classes, activation='softmax'))
        if verbose:
            model.summary()
        return model
    def train(self):
        best_val_acc = 0
        best_f=None
        best_k= None
        initial_optimizer_config = self.optimizer.get_config()
        with tqdm(total=len(self.filters) * len(self.kernels), desc="Training Progress", unit="iteration") as pbar:
            for f in self.filters:
                for k in self.kernels:
                    model = self.__create_model(f, k, self.n_classes, verbose=0)
                    optimizer = type(self.optimizer).from_config(initial_optimizer_config)
                    model.compile(optimizer=optimizer, loss=self.loss, metrics=self.metrics)
                    history = model.fit(self.X_train,self.y_train, epochs=self.epochs, batch_size=self.batch_size, validation_data=(self.X_val,self.y_val),callbacks=self.callbacks, verbose=0)
                    if history.history['val_accuracy'][-1] > best_val_acc:
                        # print('best val acc:', history.history['val_accuracy'][-1])
                        best_val_acc = history.history['val_accuracy'][-1]
                        model.save(f"model_{f}_{k}.h5")
                        best_f = f
                        best_k = k
                    pbar.update(1)
        best_model = tf.keras.models.load_model(f"model_{best_f}_{best_k}.h5")
        # best_model.compile(optimizer=self.optimizer, loss=self.loss, metrics=self.metrics)
        _, train_accuracy = best_model.evaluate(self.X_train,self.y_train, verbose=0)
        _, test_accuracy = best_model.evaluate(self.X_test,self.y_test, verbose=0)
        _, val_accuracy = best_model.evaluate(self.X_val,self.y_val, verbose=0)
        print(f"best_f: {best_f},best_k: {best_k}")
        print("Best model")
        best_model.summary()
        print("Train_accuracy:",train_accuracy,"Val accuracy:", val_accuracy, " Test Accuracy:", test_accuracy)

In [ ]:
CNN = CNN_1D(X_train, y_train, X_val, y_val, X_test, y_test)

Training Progress:   0%|          | 0/6 [00:00<?, ?iteration/s]/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
Training Progress: 100%|██████████| 6/6 [00:13<00:00,  2.19s/iteration]


best_f: [8],best_k: 2
Best model
Model: "sequential_103"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_127 (Conv1D)         (None, 50, 8)             568       
                                                                 
 batch_normalization_127 (B  (None, 50, 8)             32        
 atchNormalization)                                              
                                                                 
 max_pooling1d_127 (MaxPool  (None, 25, 8)             0         
 ing1D)                                                          
                                                                 
 global_average_pooling1d_1  (None, 8)                 0         
 03 (GlobalAveragePooling1D                                      
 )                                                               
                                                                 
 dense_103 (Dense) 